<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/EDA_Yfinance_MACD_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACD→ Analise das ações utilizando as médias moveis como orientação para compra e venda.

In [ ]:
# Instalando Yfinance API
!pip install plotly

In [1]:
# Importando as bibliotecas
import warnings
import numpy                as np
import pandas               as pd
import yfinance             as yf
import seaborn              as sns
import plotly.graph_objects as go

In [2]:
# Help function
# Configuração do padrão de visualização dos dados e graficos
# https://seaborn.pydata.org/tutorial.html
sns.set_theme(
    context='talk',
    style='ticks',
    palette='icefire',
    font_scale=.8,
    rc={'figure.figsize': (25, 15)}
)

warnings.filterwarnings("ignore")
     

In [3]:
# Escolhendo a ação
paper = 'PETR4.SA'

ticker = yf.Ticker(paper)

# Intervalo de tempo
df = ticker.history(period='30d')
df = df['Close']

df_ticker_daily = ticker.history(period='1d', interval='5m')

df_ticker_daily = df_ticker_daily['Close']

PETR4.SA: No data found for this date range, symbol may be delisted


In [4]:
info = ticker.info
acao = info['symbol']
acao

'PETR4.SA'

In [5]:
df = pd.DataFrame(df)
df_ticker_daily = pd.DataFrame(df_ticker_daily)

In [6]:
df_ticker_daily.reset_index(inplace=True)

# Quando aplicamos o Yfinance, ele acaba trazendo as informações do momento atual
# e neste momento não queremos isso
#df.drop(df.tail(1).index,inplace=True)
df.reset_index(inplace=True)

In [7]:
df

,Date,Close
0,2023-04-05 00:00:00-03:00,21.671972
1,2023-04-06 00:00:00-03:00,21.360466
2,2023-04-10 00:00:00-03:00,21.814377
3,2023-04-11 00:00:00-03:00,22.837898
4,2023-04-12 00:00:00-03:00,23.007002
5,2023-04-13 00:00:00-03:00,23.167206
6,2023-04-14 00:00:00-03:00,23.407511
7,2023-04-17 00:00:00-03:00,23.772417
8,2023-04-18 00:00:00-03:00,24.377632
9,2023-04-19 00:00:00-03:00,23.594416


In [8]:
# Criando um novo Df com os dados historicos
df_ticker = df

In [9]:
# Remove a ultima data do df_daily
df_remove_ticket = df_ticker.loc[(df_ticker['Date'] == pd.to_datetime('today').normalize())]
df_ticker = df_ticker.drop(df_remove_ticket.index)
df_ticker

,Date,Close
0,2023-04-05 00:00:00-03:00,21.671972
1,2023-04-06 00:00:00-03:00,21.360466
2,2023-04-10 00:00:00-03:00,21.814377
3,2023-04-11 00:00:00-03:00,22.837898
4,2023-04-12 00:00:00-03:00,23.007002
5,2023-04-13 00:00:00-03:00,23.167206
6,2023-04-14 00:00:00-03:00,23.407511
7,2023-04-17 00:00:00-03:00,23.772417
8,2023-04-18 00:00:00-03:00,24.377632
9,2023-04-19 00:00:00-03:00,23.594416


In [10]:
## Coletando o ultimo preço negociado até o agora
#df_ticker_daily_last_price = df_ticker_daily.tail(1)
#df_ticker_daily_last_price

In [11]:
## Ajustar o formato da data e nome coluna
#df_ticker_daily_last_price.rename(columns={'Datetime':'Date'}, inplace=True)
#df_ticker_daily_last_price

In [12]:
#df_ticker_daily_last_price_formated = pd.to_datetime(df_ticker_daily_last_price['Date'], format='%Y-%m-%d')

In [13]:
# Append data atual
#df_ticker = df_ticker.append(df_ticker_daily_last_price)
df_ticker

,Date,Close
0,2023-04-05 00:00:00-03:00,21.671972
1,2023-04-06 00:00:00-03:00,21.360466
2,2023-04-10 00:00:00-03:00,21.814377
3,2023-04-11 00:00:00-03:00,22.837898
4,2023-04-12 00:00:00-03:00,23.007002
5,2023-04-13 00:00:00-03:00,23.167206
6,2023-04-14 00:00:00-03:00,23.407511
7,2023-04-17 00:00:00-03:00,23.772417
8,2023-04-18 00:00:00-03:00,24.377632
9,2023-04-19 00:00:00-03:00,23.594416


In [14]:
#Ajustando a data
df_ticker['Date'] = pd.to_datetime(df_ticker['Date'], utc=True).dt.date
df_ticker

,Date,Close
0,2023-04-05,21.671972
1,2023-04-06,21.360466
2,2023-04-10,21.814377
3,2023-04-11,22.837898
4,2023-04-12,23.007002
5,2023-04-13,23.167206
6,2023-04-14,23.407511
7,2023-04-17,23.772417
8,2023-04-18,24.377632
9,2023-04-19,23.594416


In [15]:
# Calculando o MACD
rapidaMME=df_ticker.Close.ewm(span=12).mean()
lentaMME=df_ticker.Close.ewm(span=26).mean()

MACD = rapidaMME - lentaMME

sinal = MACD.ewm(span=9).mean()

df_ticker['MACD'] = MACD
df_ticker['sinal'] = sinal
df_ticker

,Date,Close,MACD,sinal
0,2023-04-05,21.671972,0.000000,0.000000
1,2023-04-06,21.360466,-0.006989,-0.003883
2,2023-04-10,21.814377,0.005170,-0.000172
3,2023-04-11,22.837898,0.047528,0.015986
4,2023-04-12,23.007002,0.076273,0.033920
5,2023-04-13,23.167206,0.098969,0.051552
6,2023-04-14,23.407511,0.122706,0.069559
7,2023-04-17,23.772417,0.154004,0.089853
8,2023-04-18,24.377632,0.203361,0.116074
9,2023-04-19,23.594416,0.195072,0.133774


In [16]:
# Codigo para decidiar compra e venda
df_ticker.at[0, 'flag'] = ''
df_ticker['preco_compra']=np.nan
df_ticker['preco_venda']=np.nan


for i in range(1, len(df_ticker.sinal)):
  if df_ticker['MACD'].iloc[i] > df_ticker['sinal'].iloc[i]:
    if df_ticker['flag'].iloc[i-1] == 'C':
      df_ticker['flag'][i] = 'C'
    else:
      df_ticker['flag'][i] = 'C'
      df_ticker['preco_compra'][i] = df_ticker['Close'][i]

  elif df_ticker['MACD'].iloc[i] < df_ticker['sinal'].iloc[i]:
    if df_ticker['flag'].iloc[i-1] == 'V':
      df_ticker['flag'][i] = 'V'
    else:
      df_ticker['flag'][i] = 'V'
      df_ticker['preco_venda'][i] = df_ticker['Close'][i]

df_ticker


,Date,Close,MACD,sinal,flag,preco_compra,preco_venda
0,2023-04-05,21.671972,0.000000,0.000000,,NaN,NaN
1,2023-04-06,21.360466,-0.006989,-0.003883,V,NaN,21.360466
2,2023-04-10,21.814377,0.005170,-0.000172,C,21.814377,NaN
3,2023-04-11,22.837898,0.047528,0.015986,C,NaN,NaN
4,2023-04-12,23.007002,0.076273,0.033920,C,NaN,NaN
5,2023-04-13,23.167206,0.098969,0.051552,C,NaN,NaN
6,2023-04-14,23.407511,0.122706,0.069559,C,NaN,NaN
7,2023-04-17,23.772417,0.154004,0.089853,C,NaN,NaN
8,2023-04-18,24.377632,0.203361,0.116074,C,NaN,NaN
9,2023-04-19,23.594416,0.195072,0.133774,C,NaN,NaN


In [18]:
#Visualização grafica
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [20]:
df_plot = df_ticker

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['Close'],
                         name="Preço fechamento",
                         line_color='orange'
                         ))

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['preco_compra'],
                         name="Compra",
                         mode='markers',
                         marker=  dict(
                            color='green',
                            size=10,
                            )
                         ))

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['preco_venda'],
                         name="Venda",
                         mode='markers',
                         marker=  dict(
                            color='red',
                            size=10,
                            )
                         ))

fig.show()